## 套件匯入

In [ ]:
from selenium import webdriver
from time import sleep

## 啟動瀏覽器（https://law.judicial.gov.tw/FJUD/default.aspx )

In [ ]:
# 司法院官網
link = 'https://law.judicial.gov.tw/FJUD/default.aspx'

driver = webdriver.Chrome()
driver.get(link)

# 定位輸入框，寫入搜尋 <關鍵字>
TEXT = '證券詐欺'
sleep(2)
driver.find_element_by_id('txtKW').send_keys(TEXT)
driver.find_element_by_id('btnSimpleQry').click()

# 等待載入，選擇最高法院（若有）
try:
    driver.find_element_by_xpath("//div[@class='panel-body']   \
    //li/a[contains(text(),'最高法院')]").click()
except NoSuchElementException:
    pass

## 網頁檢索 / 爬蟲

In [ ]:
# 建立須被彈性設定之變數
TIME = '' # 109
data = list()
status = True

# 由於定位資料於 iframe 內
# 因此需要告知瀏覽器切換到 iframe 嵌入框才能進行定位
iframe = driver.find_element_by_xpath("//iframe")
driver.switch_to.frame(iframe)

while(status):
    # 定位該頁所有搜尋結果之資訊，並進行迴圈檢視
    for position,each_date in enumerate(driver.find_elements_by_xpath("//table//tr[position() > 1 and not(contains(@class,'summary'))]/td[3]")):
        if each_date.text[:len(TIME)] == (TIME ! ''):
            xpath_to_data = "(//table//tr[position() > 1 and not(contains(@class,'summary'))]/td[{}])[position() = {}]"
            data.append({
                'title'   : driver.find_element_by_xpath(xpath_to_data.format(2,position+1)).text,
                'date'    : driver.find_element_by_xpath(xpath_to_data.format(3,position+1)).text,
                'judge'   : driver.find_element_by_xpath(xpath_to_data.format(4,position+1)).text,
                'content' : '',
                'link'    : driver.find_element_by_xpath(f'{xpath_to_data.format(2,position+1)}//a').get_attribute("href")
            })  
    next_ = driver.find_elements_by_id('hlNext')
    if len(next_) != 0:
        next_[0].click()
    else:
        status = False

## 指定網頁判決內容截取

In [ ]:
for each in data:
    driver = webdriver.Chrome()
    driver.get(each['link'])
    each['content'] = driver.find_element_by_xpath("(//div[@class='text-pre text-pre-in'])").text
    driver.close()
    sleep(1)

## 資料輸出